# Training, Testing & Evaluating Models for Prediction of Sales Based on City and Price of Product

In [1]:
import pandas as pd
import os

ModuleNotFoundError: No module named 'pandas'

In [ ]:
df = pd.read_csv('./datasets/Sales_Data/Sales_April_2019.csv')
files = [file for file in os.listdir('./datasets/Sales_Data')]
all_months_data = pd.DataFrame()
for file in files:
    df = pd.read_csv('./datasets/Sales_Data/'+file)
    all_months_data = pd.concat([all_months_data, df])

all_months_data.to_csv('./datasets/all_months_data.csv', index=False)

In [ ]:
all_months_data.head()